In [14]:
import pandas as pd
import numpy as np

from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [15]:
!wget 'https://drive.google.com/uc?export=download&id=1Ro6L2eRjWBIvnL49gOCkJoVAAkZOmIXZ' -O train_case2.csv

--2022-12-20 09:32:41--  https://drive.google.com/uc?export=download&id=1Ro6L2eRjWBIvnL49gOCkJoVAAkZOmIXZ
Resolving drive.google.com (drive.google.com)... 142.251.163.138, 142.251.163.102, 142.251.163.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.163.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mll77o0kcndlojsn098bpb7s8e0jmhae/1671528750000/07386159315278530712/*/1Ro6L2eRjWBIvnL49gOCkJoVAAkZOmIXZ?e=download&uuid=29862b24-554b-4106-b461-12d4fc0b0946 [following]
--2022-12-20 09:32:42--  https://doc-0c-8k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/mll77o0kcndlojsn098bpb7s8e0jmhae/1671528750000/07386159315278530712/*/1Ro6L2eRjWBIvnL49gOCkJoVAAkZOmIXZ?e=download&uuid=29862b24-554b-4106-b461-12d4fc0b0946
Resolving doc-0c-8k-docs.googleusercontent.com (doc-0c-8k-docs.googleusercontent.com)... 172.253.115.132, 

In [16]:
df = pd.read_csv('train_case2.csv', sep=';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [17]:
# разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='cardio'), 
                                                    df['cardio'], random_state=0)

In [18]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]
    
from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))])
    base_transformers.append((base_col, base_transformer))

In [19]:
# объединим все трансформеры с помощью FeatureUnion
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

**1-2. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д (2-3 варианта). При обучении моделей обязательно использовать кроссвалидацию**

***Модель логистической регрессии:***

In [20]:
model_logreg = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42))])

#запустим кросс-валидацию
cv_scores = cross_val_score(model_logreg, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score: {cv_score:.3f}+-{cv_score_std:.3f}')

#обучим пайплайн на всем тренировочном датасете
model_logreg.fit(X_train, y_train)
y_score = model_logreg.predict_proba(X_test)[:, 1]

CV score: 0.787+-0.009


In [21]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

roc_auc = roc_auc_score(y_test, y_score)

print('Best Threshold=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc_Auc Score=%.3f' % (thresholds[ix], 
                                                                                            fscore[ix],
                                                                                            precision[ix],
                                                                                            recall[ix],
                                                                                            roc_auc))

Best Threshold=0.387, F-Score=0.730, Precision=0.647, Recall=0.838, Roc_Auc Score=0.784


In [24]:
metrics_df = pd.DataFrame(columns=['Model', 'Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC'])

In [25]:
metrics_df = metrics_df.append({
    'Model': type(model_logreg['classifier']).__name__,
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035


***Модель градиентного бустинга:***

In [26]:
model_gb = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42))])

#запустим кросс-валидацию
cv_scores = cross_val_score(model_gb, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score: {cv_score:.3f}+-{cv_score_std:.3f}')

#обучим пайплайн на всем тренировочном датасете
model_gb.fit(X_train, y_train)
y_score = model_gb.predict_proba(X_test)[:, 1]

CV score: 0.803+-0.007


In [27]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

roc_auc = roc_auc_score(y_test, y_score)

print('Best Threshold=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc_Auc Score=%.3f' % (thresholds[ix], 
                                                                                            fscore[ix],
                                                                                            precision[ix],
                                                                                            recall[ix],
                                                                                            roc_auc))

Best Threshold=0.395, F-Score=0.740, Precision=0.698, Recall=0.788, Roc_Auc Score=0.803


In [28]:
metrics_df = metrics_df.append({
    'Model': type(model_gb['classifier']).__name__,
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

metrics_df

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035
1,GradientBoostingClassifier,0.394947,0.740248,0.697848,0.788134,0.802615


***Модель случайного леса:***

In [29]:
model_rf = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42))])

#запустим кросс-валидацию
cv_scores = cross_val_score(model_rf, X_train, y_train, cv=16, scoring='roc_auc')
cv_score = np.mean(cv_scores)
cv_score_std = np.std(cv_scores)
print(f'CV score: {cv_score:.3f}+-{cv_score_std:.3f}')

#обучим пайплайн на всем тренировочном датасете
model_rf.fit(X_train, y_train)
y_score = model_rf.predict_proba(X_test)[:, 1]

CV score: 0.773+-0.007


In [30]:
precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)

fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)

roc_auc = roc_auc_score(y_test, y_score)

print('Best Threshold=%.3f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc_Auc Score=%.3f' % (thresholds[ix], 
                                                                                            fscore[ix],
                                                                                            precision[ix],
                                                                                            recall[ix],
                                                                                            roc_auc))

Best Threshold=0.350, F-Score=0.719, Precision=0.643, Recall=0.816, Roc_Auc Score=0.771


In [31]:
metrics_df = metrics_df.append({
    'Model': type(model_rf['classifier']).__name__,
    'Threshold': thresholds[ix],
    'F-Score': fscore[ix],
    'Precision': precision[ix],
    'Recall': recall[ix],
    'ROC AUC': roc_auc
}, ignore_index=True)

**3. Вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)**

In [32]:
metrics_df

,Model,Threshold,F-Score,Precision,Recall,ROC AUC
0,LogisticRegression,0.386937,0.730323,0.647431,0.837558,0.784035
1,GradientBoostingClassifier,0.394947,0.740248,0.697848,0.788134,0.802615
2,RandomForestClassifier,0.350000,0.718863,0.642669,0.815553,0.771037


**4. Сделать выводы о том, какая модель справилась с задачей лучше других**

Модель градиентного бустинга справилась лучше всего, т.к. у нее более сбалансированные метрики классификации и самый высокий показатель кривой Roc_Auc.